In [1]:
from tensorflow import keras
from keras.datasets import imdb

In [2]:
# 가장 자주 나타나는 단어 300개만 사용
(train_data, train_target),(test_data, test_target) = imdb.load_data(num_words=300)

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
print(train_data.shape, train_target.shape)
print(test_data.shape, test_target.shape)

(25000,) (25000,)
(25000,) (25000,)


In [4]:
from sklearn.model_selection import train_test_split
train_data, val_data, train_target, val_target = train_test_split(train_data, train_target,
                                                                  test_size=0.2, random_state=0)

In [5]:
# 리뷰 길이를 100으로 맞추기
from keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_data, maxlen=100)
print(train_seq.shape)

(20000, 100)


In [6]:
val_seq = pad_sequences(val_data, maxlen=100)
print(val_seq.shape)

(5000, 100)


In [7]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Embedding(300, 5, input_length=100))
model.add(layers.GRU(8, return_sequences=True))
model.add(layers.GRU(8))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 5)            1500      
                                                                 
 gru (GRU)                   (None, 100, 8)            360       
                                                                 
 gru_1 (GRU)                 (None, 8)                 432       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 2,301
Trainable params: 2,301
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-lstm.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(train_seq, train_target, epochs=100,
                    validation_data=(val_seq, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
625/625 [==============================] - 14s 11ms/step - loss: 0.6051 - accuracy: 0.6506 - val_loss: 0.5258 - val_accuracy: 0.7418
Epoch 2/100
625/625 [==============================] - 6s 10ms/step - loss: 0.5091 - accuracy: 0.7534 - val_loss: 0.5093 - val_accuracy: 0.7516
Epoch 3/100
625/625 [==============================] - 6s 10ms/step - loss: 0.4976 - accuracy: 0.7599 - val_loss: 0.5030 - val_accuracy: 0.7518
Epoch 4/100
625/625 [==============================] - 6s 10ms/step - loss: 0.4919 - accuracy: 0.7654 - val_loss: 0.5040 - val_accuracy: 0.7486
Epoch 5/100
625/625 [==============================] - 6s 10ms/step - loss: 0.4882 - accuracy: 0.7660 - val_loss: 0.5002 - val_accuracy: 0.7464
Epoch 6/100
625/625 [==============================] - 6s 10ms/step - loss: 0.4869 - accuracy: 0.7651 - val_loss: 0.5040 - val_accuracy: 0.7498
Epoch 7/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4834 - accuracy: 0.7678 - val_loss: 0.5015 - val_accuracy

In [10]:
test_seq = pad_sequences(test_data, maxlen=100)
model.evaluate(test_seq[:100], test_target[:100])

4/4 [==============================] - 0s 7ms/step - loss: 0.4600 - accuracy: 0.7800


[0.4599885642528534, 0.7799999713897705]